In [9]:
from models.adapters.model_adapter_factory import create_adapter
import torch
from experiments.planner import CGPPinPlanner
from models.quantization import dequantize_per_tensor, conv2d_selector, tensor_iterator

In [10]:
adapter = create_adapter("qat_quantized_lenet", "./trained_models/qat.quantized_lenet.pth")

c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\Majo\.conda\envs\masters\Lib\site-packages\torch\ao\quantization\observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [19]:
w_train, b = adapter.get_train_weights("conv1"), adapter.get_bias("conv1")
w, b = adapter.get_weights("conv1"), adapter.get_bias("conv1")

plan = CGPPinPlanner()
plan.add_mapping(conv2d_selector("conv1", [slice(None), 0], 5, 3))
plan.finish_mapping()

x = torch.zeros(16 * 6)
for combinational_plans in plan.get_plan():
    offset = 0
    for p in combinational_plans:
        for w, size, sel in tensor_iterator(w_train, p.out):
            x[offset:offset+size] = 42
            offset+=size

x = adapter.inject_weights([x], plan.get_plan())
x.get_train_weights("conv1")

tensor([[[[  42,   42,   42,   42,   42],
          [  42,  -42,  -36,   64,   42],
          [  42,   19,   67,  127,   42],
          [  42,   60,   54,   16,   42],
          [  42,   42,   42,   42,   42]]],


        [[[  42,   42,   42,   42,   42],
          [  42,  -21,  -18,  -11,   42],
          [  42,  -25,  -71,  -69,   42],
          [  42,  -83,  -77,  -72,   42],
          [  42,   42,   42,   42,   42]]],


        [[[  42,   42,   42,   42,   42],
          [  42,  104,  106,  127,   42],
          [  42,   44,   93,  101,   42],
          [  42, -123,  -66,  -66,   42],
          [  42,   42,   42,   42,   42]]],


        [[[  42,   42,   42,   42,   42],
          [  42,  118,   38,   30,   42],
          [  42,   82,   18,  -18,   42],
          [  42,   25,  -63,  -72,   42],
          [  42,   42,   42,   42,   42]]],


        [[[  42,   42,   42,   42,   42],
          [  42,   25,   69,  -53,   42],
          [  42,   -8,  115,   63,   42],
          [  42,  

In [6]:
adapter.get_train_weights("conv1")

tensor([[[[  26,  -61,  -64,  -60,   22],
          [ -25,  -42,  -36,   64,  102],
          [  -5,   19,   67,  127,   32],
          [  32,   60,   54,   16,  -62],
          [  -9,  -12,  -32,  -96,  -76]]],


        [[[  60,   87,   24,   -3,  -23],
          [  32,  -21,  -18,  -11,  -53],
          [ -31,  -25,  -71,  -69, -127],
          [ -26,  -83,  -77,  -72,  -92],
          [ -35,  -32,  -39,  -32,   25]]],


        [[[  75,   70,  -21,  -56,  -37],
          [   7,  104,  106,  127,   51],
          [ -44,   44,   93,  101,  101],
          [ -88, -123,  -66,  -66,   25],
          [ -34,  -77, -124, -112,  -33]]],


        [[[  28,   11,   58,    6,  -44],
          [  21,  118,   38,   30,   -4],
          [ 127,   82,   18,  -18,  -81],
          [ 118,   25,  -63,  -72,  -57],
          [  66,  -11,  -91,  -79,  -61]]],


        [[[   5,   65,   -3,  -27,  -90],
          [   4,   25,   69,  -53, -127],
          [ -17,   -8,  115,   63,   16],
          [ -22,  